# Projeto - Mineração de Texto e Web
## Residência Engenharia e Ciência de dados - Samsung/UFPE

### Lucas Couri - lncc2
### Mariama Oliveira - mcso

## Pré-processamento (com e sem stemming)

# Classificadores 

## Random Forest com BoW (https://www.kaggle.com/blurredmachine/bag-of-words-meets-random-forest)

## Redes (CNN, LSTM e BERT)

## Métricas de avaliação: F1, precision e recall